In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import pandas as pd
import re
import os

path = "/content/drive/MyDrive/datasets/fakenewsnet/"

pf_fake = pd.read_csv(path + "PolitiFact_fake_news_content.csv")
pf_real = pd.read_csv(path + "PolitiFact_real_news_content.csv")
bf_fake = pd.read_csv(path + "BuzzFeed_fake_news_content.csv")
bf_real = pd.read_csv(path + "BuzzFeed_real_news_content.csv")

pf_fake["label"] = 0
bf_fake["label"] = 0
pf_real["label"] = 1
bf_real["label"] = 1


In [6]:

df = pd.concat([pf_fake, pf_real, bf_fake, bf_real], ignore_index=True)

print("Columns:", df.columns)
df.head()


Columns: Index(['id', 'title', 'text', 'url', 'top_img', 'authors', 'source',
       'publish_date', 'movies', 'images', 'canonical_link', 'meta_data',
       'label'],
      dtype='object')


,id,title,text,url,top_img,authors,source,publish_date,movies,images,canonical_link,meta_data,label
0,Real_1-Webpage,Trump Just Insulted Millions Who Lost Everythi...,16.8k SHARES SHARE THIS STORY\n\nHillary Clint...,http://occupydemocrats.com/2016/09/27/trump-ju...,http://occupydemocrats.com/wp-content/uploads/...,"Brett Bose,Grant Stern,Steve Bernstein,Natalie...",http://occupydemocrats.com,{'$date': 1474934400000},NaN,http://occupydemocrats.com/wp-content/uploads/...,http://occupydemocrats.com/2016/09/27/trump-ju...,"{""generator"": ""Powered by Visual Composer - dr...",0
1,Real_10-Webpage,Famous dog killed in spot she waited a year fo...,Famous dog killed in spot she waited a year fo...,http://rightwingnews.com/top-news/famous-dog-k...,http://rightwingnews.com/wp-content/uploads/20...,NaN,http://rightwingnews.com,{'$date': 1474948336000},NaN,http://rightwingnews.com/wp-content/uploads/20...,http://rightwingnews.com/top-news/famous-dog-k...,"{""googlebot"": ""noimageindex"", ""og"": {""site_nam...",0
2,Real_100-Webpage,House oversight panel votes Clinton IT chief i...,Story highlights The House Oversight panel vot...,http://cnn.it/2deaH2d,http://i2.cdn.cnn.com/cnnnext/dam/assets/16091...,"Tom Lobianco,Deirdre Walsh",http://cnn.it,NaN,NaN,http://i2.cdn.cnn.com/cnnnext/dam/assets/17050...,http://www.cnn.com/2016/09/22/politics/bryan-p...,"{""description"": ""Members of the House Oversigh...",0
3,Real_101-Webpage,America Just Tragically Lost A Country Music I...,We are absolutely heartbroken to hear about th...,http://newsbake.com/entertainment-news/music-e...,http://newsbake.com/wp-content/uploads/2016/05...,Nancy Wells,http://newsbake.com,{'$date': 1474898600000},https://www.youtube.com/embed/8ozTJcu-_BU,http://0.gravatar.com/avatar/0d702c6042933cd78...,http://newsbake.com/entertainment-news/music-e...,"{""shareaholic"": {""site_name"": ""NewsBake"", ""lan...",0
4,Real_102-Webpage,Monuments to the Battle for the New South,"Nine years ago, a driver lost control of his p...",http://politi.co/2dd9U1x,http://static.politico.com/25/ed/85332de14c45b...,"Jack Shafer,Lisa Rab",http://politi.co,{'$date': 1473941820000},NaN,http://static.politico.com/25/ed/85332de14c45b...,http://www.politico.com/magazine/story/2016/09...,"{""description"": ""Virginia, increasingly divers...",0


In [8]:
df["full_text"] = (df["title"].fillna("") + " " + df["text"].fillna("")).str.strip()


def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^a-zA-Z0-9\s']", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["full_text"] = df["full_text"].apply(clean_text)


df_cleaned = df[["full_text", "label"]].rename(columns={"full_text": "text"})


save_path = "/content/drive/MyDrive/misinfo_project/"
os.makedirs(save_path, exist_ok=True)

df_cleaned.to_csv(os.path.join(save_path, "fakenewsnet_cleaned.csv"), index=False)

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/drive/MyDrive/misinfo_project/data/fakenewsnet_cleaned.csv")
df.head()

X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"], test_size=0.2, random_state=42, stratify=df["label"]
)

train_df = pd.DataFrame({"text": X_train, "label": y_train})
test_df = pd.DataFrame({"text": X_test, "label": y_test})


train_df.to_csv("/content/drive/MyDrive/misinfo_project/data/fakenewsnet_train.csv", index=False)
test_df.to_csv("/content/drive/MyDrive/misinfo_project/data/fakenewsnet_test.csv", index=False)


